In [2]:
import numpy as np

In [150]:
class GaussianNB:
    # ref: https://iq.opengenus.org/gaussian-naive-bayes
    kelas = None # np.array
    jumlah_kelas = 0 # int
    jumlah_baris = 0 # int
    jumlah_fitur = 0 # int

    rataan = None # np.array (n_fitur, n_kelas)
    varian = None # np.array (n_fitur, n_kelas)
    target = None # np.array (n_kelas,)

    def __init__(self):
        print("[INFO] Model terinisialisasi!")

    def fit(self, X:np.array, y:np.array):
        self.kelas = np.unique(y)
        self.jumlah_kelas = len(self.kelas)
        self.jumlah_fitur = len(X[0])
        self.__setup_rataan_varian_target__(X, y)
        print("[INFO] Model sudah berhasil belajar")

    def predict(self, X:np.array) -> np.array:
        hasil = np.zeros((len(X), ))
        for i in range(len(X)):
            hasil[i] = self.kelas[self.__get_kelas__(X[i])]
        return hasil

    def __setup_rataan_varian_target__(self, X: np.array, y:np.array):
        '''
        Hitung Probabilitas Fitur terhadap Target (P(Xi, Xi+1, Xi+2, ... Xn))
        Hitung Probabilitas Target (P(yi))
        ''' 
        ## Setup group by 
        kelas = {}
        for i in self.kelas:
            kelas[i] = []
        for i in range(len(y)):
            if y[i] in kelas.keys():
                kelas[y[i]].append(i)
        
        # Setup probabilitas fitur terhadap target
        self.rataan = np.array([np.zeros((self.jumlah_fitur, )) for i in range(self.jumlah_kelas)])
        self.varian = np.array([np.zeros((self.jumlah_fitur, )) for i in range(self.jumlah_kelas)])
        j = 0
        for i in kelas.keys():
            sum = np.sum(X[kelas[i]], axis=0)
            self.rataan[j] = sum/len(kelas[i])
            var = np.var(X[kelas[i]], axis=0)
            self.varian[j] = var
            j += 1

        # Setup probabilitas target
        self.target = np.zeros((self.jumlah_kelas,))
        j = 0
        for i in kelas.keys():
            self.target[j] = len(kelas[i]) / len(y)
            j += 1
    
    def __get_kelas__(self, X: np.array) -> int:
        '''
        X (n_fitur, )
        '''
        prob = []
        for i in range(len(self.kelas)):
            prob_kelas = np.log(self.target[i])
            prob_fitur = np.sum(np.log(np.exp((-1/2)*((X-self.rataan[i])**2) / (2 * self.varian[i])) / np.sqrt(2 * np.pi * self.varian[i])))
            prob.append(prob_kelas + prob_fitur)
        return np.argmax(prob)

# Made from love, by Kaenova and Moh. Adi Ikfini :)